**Titanic Whithout Using Pipelines**

In [79]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [80]:
df = pd.read_csv('titanic.csv')

In [81]:
df.head()

PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]

In [82]:
df.shape

(891, 12)

In [83]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [84]:
df.sample(5)

Survived  Pclass     Sex   Age  SibSp  Parch      Fare Embarked
52          1       1  female  49.0      1      0   76.7292        C
775         0       3    male  18.0      0      0    7.7500        S
798         0       3    male  30.0      0      0    7.2292        C
311         1       1  female  18.0      2      2  262.3750        C
186         1       3  female   NaN      1      0   15.5000        Q

In [85]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [86]:
x_train.head()

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
331       1    male  45.5      0      0  28.5000        S
733       2    male  23.0      0      0  13.0000        S
382       3    male  32.0      0      0   7.9250        S
704       3    male  26.0      1      0   7.8542        S
813       3  female   6.0      4      2  31.2750        S

In [87]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [88]:
# appling imputation
si_age =SimpleImputer()#age missing value is filled with mean
si_embarked = SimpleImputer(strategy='most_frequent')#fill the most frequent value : S

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])

In [89]:
# onehot encoding sex and embarked

ohe_sex = OneHotEncoder(handle_unknown='ignore')
ohe_embarked = OneHotEncoder(handle_unknown = 'ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)

# handle_unknown = 'ignore' -> In future if we get any other category in test we will ignore it

In [90]:
x_train_sex.toarray()

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [91]:
x_train.head(2)

Pclass   Sex   Age  SibSp  Parch  Fare Embarked
331       1  male  45.5      0      0  28.5        S
733       2  male  23.0      0      0  13.0        S

In [92]:
x_train_rem = x_train.drop(columns = ['Sex','Age','Embarked'])
x_test_rem = x_test.drop(columns = ['Sex','Age','Embarked'])

In [93]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex.toarray(),x_train_embarked.toarray()),axis=1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex.toarray(),x_test_embarked.toarray()),axis=1)

In [94]:
x_train_transformed

array([[1., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 2., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [95]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [96]:
y_pred = clf.predict(x_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1])

In [97]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

In [98]:
import pickle

In [101]:
# pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))//for converting data into numnerical data
# pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))// for converting categorical data into numerical data
# pickle.dump(clf,open('models/clf.pkl','wb'))//classifier will prdict the output

In [102]:
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))